In [95]:
import numpy as np
import os, re, csv, math, codecs
from keras.models import Sequential
from keras.layers import Embedding
from keras.preprocessing import sequence
from tqdm import tqdm
from keras.layers import merge
from keras.layers.core import *
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import *
# to visualize, and to make zero shape matrix
from attention_utils import get_activations, get_data_recurrent
# from Attention import attention
import pandas as pd

from konlpy.tag import Okt as Twitter



# 1. 모델 저장시키기
# 2. tokenizer와 konlpy morphs 호환 여부
# 3. 변수들 설정하기


np.random.seed(0)

# NUM_WORDS, train으로 input받은 단어의 수
MAX_NB_WORDS = 20000 
EMB_DIM = 300
embeddings_index = dict()
# INPUT 문장길이
text_max_words = 280

# columns = ["ID", "Tweet", "anger", "anticipation", "disgust", "fear", "joy", "love", "optimism", "pessimism","sadness","surprise","trust"]
columns = ["ID","Tweet","분노","기대","혐오스러운","두려움","기쁨","사랑","낙관론","비관론","슬픔","놀라움","믿음"]



# def parse(dataset, emotion=None):
#     with open(data_file, 'r') as fd:
#         data = [l.strip().split('\t') for l in fd.readlines()][1:]
#     X = [d[1] for d in data]
#     # dict.values() does not guarantee the order of the elements
#     # so we should avoid using a dict for the labels
#     y = [[int(l) for l in d[2:]] for d in data]

#     return X, y


##########################################1. 데이터셋 생성하기

    # X_train, y_train = parse(task='E-c', dataset="train")
    # X_dev, y_dev = parse(task='E-c', dataset="dev")
    # X_test, y_test = parse(task='E-c', dataset="gold")
    # X_train = preprocessor("{}_{}".format(name, "train"), X_train)
    # X_dev = preprocessor("{}_{}".format(name, "dev"), X_dev)
    # X_test = preprocessor("{}_{}".format(name, "test"), X_test)
    # res_dev = fit_multilabel(model, X_train, X_dev, y_train, y_dev)
    # res_test = fit_multilabel(model, X_train, X_test, y_train, y_test)

    # res_dev = evaluation(res_dev, y_dev)
    # res_test = evaluation(res_test, y_test)

# train_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_train.txt",sep="\t", header=None,names=columns).values
val_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_dev.txt",sep="\t", header=None,names=columns).values
# test_array = pd.read_csv("/home/minwookje/coding/ex1_w2v/data/tweet/dump/kor_test.txt",sep="\t", header=None,names=columns).values

# print(train_array.shape)
# print(val_array.shape)
# print(test_array.shape)

#  x, y 분할하기
# x_train = train_array[:,1]
# y_train = train_array[:,2:]
x_val = val_array[1:,1]
y_val = val_array[1:,2:]
# x_test = test_array[:,1]
# y_test = test_array[:,2:]


twitter = Twitter()
tmp = []
max_count = 0
set_words = set()
for i,x in enumerate(x_val):
    x = re.sub(r"[^ㄱ-힣a-zA-Z0-9]+", ' ', x).strip().split()
# RuntimeError: No matching overloads found. at src/native/common/jp_method.cpp:121
# 에러가 뜨는 경우에는 argument를 잘 확인해 주어야 한다. 현재의 경우 str이 들어가야 하는 자리에 list형식이 들어가있다.
    tmp.append(twitter.pos(str(x), norm=True, stem=True))
    
#      if twitter.pos(str(x))[1] is not 'Punctuation')
#     tmp = [s for s in tmp[i] if (s[1] != 'Punctuation' or s[1] != 'Alpha')]
#     (b.append(a) if a is not None else None)
    for j in tmp[i]:
        set_words.add(j)
    if len(tmp[i]) > max_count:
        max_count = len(tmp[i])
        if max_count == 93:
            print(i)
        
print(max_count)
    

# # Tokenize
# tokenizer = Tokenizer(num_words=max_features)
# tokenizer.fit_on_texts(list(list_sentences))
# list_tokenized_train = tokenizer.texts_to_sequences(list_sentences)

# # Pad
# x = pad_sequences(list_tokenized_train, maxlen=maxlen)


# x_val = sequence.pad_sequences(x_val, maxlen=text_max_words)


613
93


In [98]:
print(x_val[613])
print(tmp[613])
print(len(tmp[613]))

나는 너를 잃을 수 없다. 왜냐하면 내가 한 적이 있다면, 나는 내 가장 친한 친구, 내 영혼의 짝, 나의 미소, 내 웃음, 내 모든 것을 잃어 버렸기 때문이다.
[("['", 'Punctuation'), ('나', 'Noun'), ('는', 'Josa'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('너', 'Noun'), ('를', 'Josa'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('잃다', 'Verb'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('수', 'Noun'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('없다', 'Adjective'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('왜냐하면', 'Adverb'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('내', 'Noun'), ('가', 'Josa'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('하다', 'Verb'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('적', 'Noun'), ('이', 'Josa'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('있다', 'Adjective'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('나', 'Noun'), ('는', 'Josa'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('내', 'Noun'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('가장', 'Noun'), ("',", 'Punctuation'), ("'", 'Punctuation'), ('친하다', 'A

In [51]:
import nltk
dummy_nltk = nltk.sent_tokenize(x_val)

LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/PY3/english.pickle' not found.
  Please use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - '/home/minwookje/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************

In [52]:
def Convert2Vec(self, model_name, doc):  ## Convert corpus into vectors
#     word_vec = []
#     model = gensim.models.word2vec.Word2Vec.load(model_name)
    for sent in doc:
        sub = []
        for word in sent:
            if(word in model.wv.vocab):
                sub.append(model.wv[word])
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) ## used for OOV words
        word_vec.append(sub)
    
    return np.array(word_vec)

In [2]:


# tmp1= 'a'
# tmp2= 'a'
# for i in a:
#     tmp1 = i
#     tmp2 = str(i)
#     print(i,str(i))
#     break
# if str(tmp1) == tmp2:
#     print("yes!")
#     쌍콤마 지우기 코드, 필요없을 수도 str씌우니까 dic_key타입이 ""사라지게 되는데
# for i in :
#     embeddings_index[i[1:-1]] = embeddings_index.pop(i)


14241it [00:01, 8401.33it/s]

Loaded 14241 word vectors.


In [5]:
k = embeddings_index
print(type(k))

<class 'dict'>


In [70]:
print(embeddings_index["('대통령','Noun')"])

[-0.209341    0.0262855  -0.16035201  0.19948401  0.582286   -0.0345117
 -0.121298   -0.0912677  -0.51300198  0.27131999  0.0968003   0.0865747
  0.17522299 -0.36303499 -0.319419    0.080253   -0.245655    0.170451
  0.273516   -0.253815   -0.17695899  0.207268    0.276824    0.0145649
 -0.19889501  0.21917699  0.127717   -0.49855    -0.0957409  -0.91694498
  0.0827512  -0.161661   -0.00689074 -0.0747906   0.27707499  0.00543082
  0.15478399  0.181251   -0.44859901  0.18153299  0.628097   -0.0156562
  0.088979    0.310588    0.124403   -0.080413    0.156323   -0.134864
  0.23675799 -0.00917961  0.12836     0.29467499 -0.170986   -0.201645
 -0.00965752 -0.103357   -0.00537147  0.25499001 -0.0436398   0.25271299
  0.058608   -0.134019    0.184654    0.46035799  0.0289141  -0.15943199
 -0.57392198 -0.38885701 -0.303141    0.12956899 -0.33357501 -0.0230292
 -0.25253001  0.34411901 -0.13720299 -0.0360019  -0.54765701  0.26774201
 -0.145886    0.0787636   0.106609    0.23292901 -0.126531   -

AttributeError: type object 'Tokenizer' has no attribute 'word_index'

In [72]:
f = codecs.open('/home/minwookje/coding/ex1_w2v/embedding/1542948553only_go_twitter_pos.vec', encoding='utf-8')
for line in tqdm(f):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


# word2index table 생성
# train 문장 token 갯수
word_index = len(set_words)
word_tmp_dict = dict()
for i, word in enumerate(set_words):
    word_tmp_dict[str(word).replace(" ", "")] = i
word_tmp_dict['0'] = word_index 

## 문장별 토큰화시킨 녀석에 index를 집어 넣어준다. 이때 pad도 동시에 해준다. 

word_vec = []
for sent in tmp:
    sub = []
    for word in sent:
         #print(word)
         #print(type(str(word)))
         #break
# word는 tuple 타입, embeddings_index는 str타입, tuple 타입을 str()화시키면 
# 중간에 space가 생성되어 match가 되지 않았다. 이를 해결해주었다. 
        if(str(word).replace(" ", "") in word_tmp_dict):
            
            sub.append(word_tmp_dict[str(word).replace(" ", "")])
        else:
            print("sentence index화 실패")
    count = max_count - len(sub)
    sub.extend([word_index]*count)
    word_vec.append(sub)


# 4번쨰 matrix
vocab_size = min(MAX_NB_WORDS, word_index)
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size+1, EMB_DIM))
match_count = 0
unmatch_count = 0

for word, i in word_tmp_dict.items():
    if word != '0':
        if (word in embeddings_index):
            match_count += 1
            embedding_matrix[i] = embeddings_index[word]
        else:
            unmatch_count += 1
            embedding_matrix[i] = np.random.uniform(-0.25,0.25,300) ## used for OOV words




    
#     여기부터
# 1.앞서 단어당 벡터 테이블(v) // embeddings_index, {w2v_word: vector}
# train_word(str(word).replace(" ", "")) == embedding
# 2.train 단어별 index (v) // word_tmp_dict {train_word(str(word).replace(" ", "")):index}
# 3.sentence padding, sentence to index
# 4.index당 vector table (x) //embedding_matrix {index: vector} 이녀석을 embedding weight에 넣어주어야 한다. 

# 문장 = [index들 나열 ] 
# 즉 embedding_matrix로 index를 seq에 넣어준묹장들을 train에 넣어줘야한다. 


# print(match_count, unmatch_count)
# print(len(tmp))
# print(x_val.shape)

# x_val =  np.array(word_vec)

14241it [00:01, 8448.76it/s]


Loaded 14241 word vectors.


In [84]:
print(len(word_vec[300]))

93


array([ 0.02440675,  0.10759468,  0.05138169,  0.02244159, -0.0381726 ,
        0.07294706, -0.03120639,  0.1958865 ,  0.23183138, -0.05827924,
        0.14586252,  0.01444746,  0.03402228,  0.21279832, -0.21448197,
       -0.20643535, -0.2398908 ,  0.16630992,  0.13907838,  0.18500607,
        0.23930917,  0.14957928, -0.01926032,  0.14026459, -0.19086279,
        0.06996051, -0.17832336,  0.22233446,  0.01092416, -0.04266903,
       -0.11772219,  0.13711684, -0.02192483,  0.03421697, -0.2406051 ,
        0.05881775,  0.05604786,  0.058467  ,  0.22187404,  0.09091015,
       -0.07024605, -0.03148402,  0.0988156 , -0.21988726,  0.08338336,
        0.08531893, -0.14480872, -0.18553685, -0.09228582, -0.06814461,
        0.03509839, -0.03069924,  0.24418692, -0.19897759, -0.14556162,
       -0.16934524,  0.07655416, -0.1233542 , -0.01684461, -0.1277872 ,
       -0.17051521, -0.19481243,  0.07816479, -0.18090852, -0.15170882,
       -0.06563741,  0.16049661, -0.20144936,  0.16897245, -0.20

In [16]:
a = [1,2,3]
a.extend([0]*4)
print(a)

[1, 2, 3, 0, 0, 0, 0]


In [35]:
lists = [tmp[1]]+ [tmp[2]]
# lists =[ ['9034968', 'ETH'], ['14160113', 'ETH'], ['9034968', 'ETH'], ['11111', 'NOT'], ['9555269', 'NOT'], ['15724032', 'ETH'], ['15481740', 'ETH'], ['15481757', 'ETH'], ['15481724', 'ETH'], ['10307528', 'ETH'], ['15481757', 'ETH'], ['15481724', 'ETH'], ['15481740', 'ETH'], ['15379365', 'ETH'], ['11111', 'NOT'], ['9555269', 'NOT'], ['15379365', 'ETH']]


In [37]:
unique_data = [x for x in set(tuple_e for tuple_e in (list_x for list_x in lists))]

TypeError: unhashable type: 'list'

In [61]:
# k= [list(x) for x in set(tuple(x) for x in [[1,2,3],['a','b','c','1']])]
k = [list(x) for x in set(tuple(x) for x in [[1,2,3],['1','b','c','1']])]
print(k)

[['1', 'b', 'c', '1'], [1, 2, 3]]


In [34]:
print(len(lists))
print(len(unique_data))

17
10
